<a href="https://colab.research.google.com/github/GNuSeekK/ICTCOG/blob/main/4_04_%5B%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_%ED%96%89%EB%A0%AC%EA%B3%BC_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망의 입력과 가중치
* 신경망에 입력 되는 값은 여러개
  * $X = [x_1, x_2, x_3, \cdots, x_n]$
* 입력값에 해당하는 가중치도 여러 개
  * $ W = [w_1, w_2, w_3, \cdots, w_n]$
  * $n$개의 입력을 받는 $m$개의 뉴런이 있다.
  * 예를 들어 2개의 입력을 받는 3개의 뉴런을 행렬로 표기해야 한다.
\begin{pmatrix}w_1 & w_2 & w_3 \\ w_4 & w_5 & w_6 \end{pmatrix}


In [ ]:
import numpy as np

X = np.array([1,2])
W = np.array([[1,2,3],
              [4,5,6]])

print(f'입력값 X의 shape : {X.shape}')
print(f'입력값 W의 shape : {W.shape}')

입력값 X의 shape : (2,)
입력값 W의 shape : (2, 3)


$X, W$의 내적이 가능하다

In [ ]:
Z = X @ W
print(f'출력값 Z : {Z}')

출력값 Z : [ 9 12 15]


[번외] 위 처럼 표기하면 $Z=XW+B$

$Z=WX+B$의 형식으로 코딩할거면 `Z = W.T @ X + B`

In [ ]:
Z = W.T @ X
print("출력값 Z : {}".format(Z))

출력값 Z : [ 9 12 15]


# 신경망 표기 법칙

$$
w^{(To)}_{(To)(From)}
$$

**1층을 구성하는 표기**

* 1층의 1번째 뉴런만 표기 : $z_1^{(1)} = w_{11}^{(1)}x_1 + w_{12}^{(1)}x_2+b_1^{(1)}$
* 1층 전체에 대해 표기 : $Z^{(1)} = XW^{(1)} + B^{(1)}$ / $Z^{(1)} = W^{(1)^{T}}X + B^{(1)}$ 
  * 편향 $B$의 원소의 개수는 뉴런의 개수와 같다.
-----
 * $X = (x_1, x_2)$
 * $W^{(1)} = \begin{pmatrix}w_{11}^{(1)} & w_{21}^{(1)} & w_{31}^{(1)} \\ w_{12}^{(1)} & w_{22}^{(1)} & w_{32}^{(1)}
 \end{pmatrix}$
 * $B^{(1)} = \begin{pmatrix}b_1^{(1)}&b_2^{(1)}&b_3^{(1)}\end{pmatrix}$
 * $Z^{(1)} = \begin{pmatrix}z_1^{(1)}&z_2^{(1)}&z_3^{(1)}\end{pmatrix}$




# 1층 구현하기
* 모든 X, W, B 는 임의로 사람이 정한 것..
* W, B를 모델이 구할 수 있도록 하는 것이 **신경망 학습**

In [ ]:
# 시그모이드 함수 구현
def sigmoid(x):
  return 1 / ( 1 + np.exp(-x) )

# 항등 함수 구현 - 출력층에서 사용할 함수..
def identity_function(x):
  return x

In [ ]:
X = np.array([1.0, 0.5])         # (2, )
W1 = np.array([[0.1, 0.3, 0.5],
               [0.2, 0.4, 0.6]]) # (2, 3)

B1 = np.array([0.1, 0.2, 0.3])   # (3, )

# 뉴런 계산
Z1 = X @ W1 + B1

# 활성화 함수 적용
A1 = sigmoid(Z1)

print("1층의 결과 : {}".format(A1))

1층의 결과 : [0.57444252 0.66818777 0.75026011]


# 2층 구현하기
* 1층의 결과물인 `A1`을 입력으로 받는 2개의 뉴런
  * `A1`의 `shape` : `(3, )`
  * `A1`을 입력으로 받을 2층의 가중치의 `shape` : `(3, 2)`

In [ ]:
W2 = np.array([[0.1, 0.4],
               [0.2, 0.5],
               [0.3, 0.6]]) # (3, 2)

B2 = np.array([0.1, 0.2])  # (2, )

Z2 = A1 @ W2 + B2
A2 = sigmoid(Z2)

print("2층의 결과 : {}".format(A2))

2층의 결과 : [0.62624937 0.7710107 ]


# 3층 구현하기 (출력층)
  * 활성화 함수로 항등함수(`identity_function`)을 사용한다.
  * 항등함수 : 입력되는 값을 그대로 리턴하면 항등함수
    * 회귀에서 보통 사용되는 출력층의 활성화 함수
    * 레이어마다 계산(wx+b)후에는 항상 활성화 함수를 붙여 줬기 때문에 출력층에도 항등함수라는 활성화 함수를 따로 지정해 주려고..

In [ ]:
W3 = np.array([[0.1, 0.3],
               [0.2, 0.4]])

B3 = np.array([0.1, 0.2])

Z3 = A2 @ W3 + B3

# 신경망의 출력층의 결과물은 Y
Y = identity_function(Z3)

print(Y)

[0.31682708 0.69627909]


# 순전파 (Forward Propagation) 최종 구현하기

In [ ]:
# 네트워크 초기화
def init_network():
  '''
  네트워크가 최초로 가지고 있어야 할 가중치와 편향을 설정
  네트워크 객체가 만들어 지면서 가지고 있어야 할 최초의 값...
  
  매개변수 ( 모델 파라미터 ) : W, b

  Tensorflow 같은 딥러닝 프레임워크에서는 3가지 방법으로 매개변수를 초기화 한다.
  1. 정규분포 랜덤 ( 표준편차 0.01 )
  2. 사비에르 글로로트 초깃값
  3. 카이밍 히 초깃값
'''
  network = {}
  # 1층 매개변수 초기화
  network['W1'] = np.array([[0.1, 0.3, 0.5],
                            [0.2, 0.4, 0.6]]) # 2개의 입력을 받는 3개의 뉴런
  
  network['b1'] = np.array([0.1, 0.2, 0.3])

  # 2층 매개변수 초기화
  network['W2'] = np.array([[0.1, 0.4],
                            [0.2, 0.5],
                            [0.3, 0.6]]) # 3개의 입력을 받는 2개의 뉴런
  
  network['b2'] = np.array([0.1, 0.2])

  # 3층 매개변수 초기화
  network['W3'] = np.array([[0.1, 0.3],
                            [0.2, 0.4]]) # 2개의 입력을 받는 2개의 뉴런
  
  network['b3'] = np.array([0.1, 0.2])

  return network


# 순전파 XW+B
def forward(network, x):
  # 가중치, 편향 꺼내기
  W1, W2, W3 = network['W1'], network['W2'], network['W3']
  b1, b2, b3 = network['b1'], network['b2'], network['b3']

  # 1층 계산
  Z1 = x @ W1 + b1
  A1 = sigmoid(Z1)

  # 2층 계산
  Z2 = A1 @ W2 + b2
  A2 = sigmoid(Z2)

  # 3층 계산

  Z3 = A2 @ W3 + b3
  y = identity_function(Z3)

  return y

간단한 네트워크 순전파만!

In [ ]:
network = init_network() # 네트워크 초기화
x = np.array([1.0,0.5])
y = forward(network,x)

print(y)

[0.31682708 0.69627909]
